In [1]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.71.1
    Uninstalling google-cloud-aiplatform-1.71.1:
      Successfully uninstalled google-cloud-aiplatform-1.71.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==1.71.1, but you have google-cloud-aiplatform 1.102.0 which is incompatible.


In [13]:
from google.cloud import aiplatform

# Set up project info
project_id = "theramind"
location = "us-central1"  # or where your bucket is
aiplatform.init(project=project_id, location=location)

In [14]:
GCS_BUCKET = "therachat"
TRAIN_FILE = f"gs://{GCS_BUCKET}/gemini_sft/gemini_train.jsonl"
VAL_FILE = f"gs://{GCS_BUCKET}/gemini_sft/gemini_val.jsonl"

In [4]:
!pip install --upgrade google-cloud-aiplatform

In [5]:
!pip install --upgrade google-cloud-aiplatform vertexai

  Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached google_cloud_aiplatform-1.71.1-py2.py3-none-any.whl (6.2 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.102.0
    Uninstalling google-cloud-aiplatform-1.102.0:
      Successfully uninstalled google-cloud-aiplatform-1.102.0


In [15]:
import vertexai
from vertexai.tuning import sft

project_id = "thera-mind"
location = "us-central1"
vertexai.init(project=project_id, location=location)

In [17]:
from google.cloud import aiplatform
import vertexai
from vertexai.tuning import sft

# Init your environment
PROJECT_ID = "thera-mind"
LOCATION = "us-central1"
BUCKET = "therachat"

aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

# File paths
TRAIN_FILE = f"gs://{BUCKET}/gemini_sft/gemini_train.jsonl"
VAL_FILE = "gs://therachat/gemini_sft/gemini_val.jsonl"

# Launch job
SOURCE_MODEL = "gemini-2.0-flash-001"
DISPLAY_NAME = "therachat-empathetic-v2"

sft_job = sft.train(
    source_model=SOURCE_MODEL,
    train_dataset=TRAIN_FILE,
    validation_dataset=VAL_FILE,
    tuned_model_display_name=DISPLAY_NAME
)

print("✅ Supervised fine-tuning job started!")
print("👉 Tune Job resource name:", sft_job.resource_name)
print("👉 Tuned model endpoint (after completion):", sft_job.tuned_model_endpoint_name)

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/996770367618/locations/us-central1/tuningJobs/4853676518972325888
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/996770367618/locations/us-central1/tuningJobs/4853676518972325888')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/4853676518972325888?project=996770367618


✅ Supervised fine-tuning job started!
👉 Tune Job resource name: projects/996770367618/locations/us-central1/tuningJobs/4853676518972325888
👉 Tuned model endpoint (after completion): 


In [13]:

print(f"Tuned model resource name: {sft_job.tuned_model_name}")
print(f"Tuned model endpoint name: {sft_job.tuned_model_endpoint_name}")

Tuned model resource name: 
Tuned model endpoint name: 


In [14]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.tuning import sft # This import is correct for SFT

# --- Initialization (make sure this is run first) ---
PROJECT_ID = "996770367618" # Your specific project ID from the job details
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

# --- Retrieve the completed job object using its Job ID ---
# This is the Job ID from the details you provided:
tuning_job_id = "7177680161741996032"

sft_job = sft.SupervisedTuningJob(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}"
)

# --- Print the tuned model and endpoint names ---
print(f"Tuned model resource name: {sft_job.tuned_model_name}")
print(f"Tuned model endpoint name: {sft_job.tuned_model_endpoint_name}")


# --- Evaluate the Tuned Model ---
# This will load the model from the endpoint created by the tuning job.
# The 'tuned_model_endpoint_name' links directly to the deployed model.
tuned_model = GenerativeModel(sft_job.tuned_model_endpoint_name)

# --- Example Evaluation ---
# IMPORTANT: Use the exact same prompt format as your training data.
# If your training data has "User: <user_message> Response: <model_response>",
# then your prompt for inference should end with " Response:" or whatever
# delimiter you used to signal where the model should start generating.

print("\n--- Evaluating Tuned Model ---")

# Prompt 1: (Example for empathetic chat)
prompt1 = "User: I'm feeling incredibly overwhelmed with work right now. Response:"
print(f"\nPrompt 1:\n{prompt1}")
response1 = tuned_model.generate_content(
    [Part.from_text(prompt1)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0 # Recommended for tuned models like Gemini 2.5 Flash if applicable
    }
)
print("Tuned Model Response:", response1.text)

# Prompt 2: (Another example)
prompt2 = "User: My best friend just moved away, and I feel so lonely. Response:"
print(f"\nPrompt 2:\n{prompt2}")
response2 = tuned_model.generate_content(
    [Part.from_text(prompt2)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0
    }
)
print("Tuned Model Response:", response2.text)

Tuned model resource name: projects/996770367618/locations/us-central1/models/5874825867137384448@1
Tuned model endpoint name: projects/996770367618/locations/us-central1/endpoints/8689516260616044544

--- Evaluating Tuned Model ---

Prompt 1:
User: I'm feeling incredibly overwhelmed with work right now. Response:
Tuned Model Response: It sounds like you're dealing with a significant amount of stress related to work. Feeling overwhelmed can be incredibly difficult. What aspects of your work are contributing the most to your stress?

Prompt 2:
User: My best friend just moved away, and I feel so lonely. Response:
Tuned Model Response: It sounds like you're experiencing a deep sense of loneliness after your best friend moved away. That kind of separation can be incredibly painful. What's been the hardest part about not having them around?


In [15]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.tuning import sft # This import is correct for SFT

# --- Initialization (make sure this is run first) ---
PROJECT_ID = "996770367618" # Your specific project ID from the job details
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

# --- Retrieve the completed job object using its Job ID ---
# This is the Job ID from the details you provided:
tuning_job_id = "7177680161741996032"

sft_job = sft.SupervisedTuningJob(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}"
)

# --- Print the tuned model and endpoint names ---
print(f"Tuned model resource name: {sft_job.tuned_model_name}")
print(f"Tuned model endpoint name: {sft_job.tuned_model_endpoint_name}")


# --- Evaluate the Tuned Model ---
# This will load the model from the endpoint created by the tuning job.
# The 'tuned_model_endpoint_name' links directly to the deployed model.
tuned_model = GenerativeModel(sft_job.tuned_model_endpoint_name)

# --- Example Evaluation ---
# IMPORTANT: Use the exact same prompt format as your training data.
# If your training data has "User: <user_message> Response: <model_response>",
# then your prompt for inference should end with " Response:" or whatever
# delimiter you used to signal where the model should start generating.

print("\n--- Evaluating Tuned Model ---")

# Prompt 1: (Example for empathetic chat)
prompt1 = "User: What is the capital of Estonia? Response:"
print(f"\nPrompt 1:\n{prompt1}")
response1 = tuned_model.generate_content(
    [Part.from_text(prompt1)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0 # Recommended for tuned models like Gemini 2.5 Flash if applicable
    }
)
print("Tuned Model Response:", response1.text)

# Prompt 2: (Another example)
prompt2 = "User: What was the formula for quadratic equation? Response:"
print(f"\nPrompt 2:\n{prompt2}")
response2 = tuned_model.generate_content(
    [Part.from_text(prompt2)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0
    }
)
print("Tuned Model Response:", response2.text)

Tuned model resource name: projects/996770367618/locations/us-central1/models/5874825867137384448@1
Tuned model endpoint name: projects/996770367618/locations/us-central1/endpoints/8689516260616044544

--- Evaluating Tuned Model ---

Prompt 1:
User: What is the capital of Estonia? Response:
Tuned Model Response: I cannot entertain this query. Please ask questions related to the text only.

Prompt 2:
User: What was the formula for quadratic equation? Response:
Tuned Model Response: I cannot entertain this query. Please ask questions related to the text only.


In [17]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.tuning import sft # This import is correct for SFT

# --- Initialization (make sure this is run first) ---
PROJECT_ID = "996770367618" # Your specific project ID from the job details
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

# --- Retrieve the completed job object using its Job ID ---
# This is the Job ID from the details you provided:
tuning_job_id = "7177680161741996032"

sft_job = sft.SupervisedTuningJob(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}"
)

# --- Print the tuned model and endpoint names ---
print(f"Tuned model resource name: {sft_job.tuned_model_name}")
print(f"Tuned model endpoint name: {sft_job.tuned_model_endpoint_name}")


# --- Evaluate the Tuned Model ---
# This will load the model from the endpoint created by the tuning job.
# The 'tuned_model_endpoint_name' links directly to the deployed model.
tuned_model = GenerativeModel(sft_job.tuned_model_endpoint_name)

# --- Example Evaluation ---
# IMPORTANT: Use the exact same prompt format as your training data.
# If your training data has "User: <user_message> Response: <model_response>",
# then your prompt for inference should end with " Response:" or whatever
# delimiter you used to signal where the model should start generating.

print("\n--- Evaluating Tuned Model ---")

# Prompt 1: (Example for empathetic chat)
prompt1 = "User: I am sufferring from headache. What medicine must I take? Response:"
print(f"\nPrompt 1:\n{prompt1}")
response1 = tuned_model.generate_content(
    [Part.from_text(prompt1)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0 # Recommended for tuned models like Gemini 2.5 Flash if applicable
    }
)
print("Tuned Model Response:", response1.text)

# Prompt 2: (Another example)
prompt2 = "User: What is stress? (since I am sufferring from it) Response:"
print(f"\nPrompt 2:\n{prompt2}")
response2 = tuned_model.generate_content(
    [Part.from_text(prompt2)],
    generation_config={
        "max_output_tokens": 200,
        "temperature": 0.7,
        "top_p": 0.9,
        # "thinking_budget": 0
    }
)
print("Tuned Model Response:", response2.text)

Tuned model resource name: projects/996770367618/locations/us-central1/models/5874825867137384448@1
Tuned model endpoint name: projects/996770367618/locations/us-central1/endpoints/8689516260616044544

--- Evaluating Tuned Model ---

Prompt 1:
User: I am sufferring from headache. What medicine must I take? Response:
Tuned Model Response: I cannot entertain this query. Please ask questions related to headache only.

Prompt 2:
User: What is stress? (since I am sufferring from it) Response:
Tuned Model Response: It sounds like you are currently experiencing stress. Can you tell me more about what you are going through?


In [19]:
from vertexai.generative_models import GenerativeModel, Part

# Assuming 'tuned_model' is your loaded Gemini model

# General mental health question
general_question_prompt = "What are some common signs of anxiety?"

# --- Method 1: Prepending an instruction ---
# This tries to guide the model's behavior for this specific query.
instructional_prompt = f"""You are a helpful and informative AI assistant specializing in mental health awareness. Please provide general information and definitions when asked about mental health topics.
User: {general_question_prompt}
Response:"""
print(f"\n--- Instructional Prompt ---\n{instructional_prompt}")
response_instructional = tuned_model.generate_content(
    [Part.from_text(instructional_prompt)],
    generation_config={"max_output_tokens": 300, "temperature": 0.5, "top_p": 0.9}
).text
print("Tuned Model Response (Instructional):", response_instructional)

# --- Method 2: Multi-turn chat context (less direct for this specific bypass) ---
# If your model supports multi-turn and you wanted to establish a persona first.
# This is more for persistent behavior across a conversation.
# For a single query, Method 1 is more direct.
# chat = tuned_model.start_chat(history=[])
# chat.send_message("From now on, please act as an informative mental health guide.")
# response_chat = chat.send_message(f"User: {general_question_prompt}\nResponse:").text
# print("Tuned Model Response (Chat Context):", response_chat)


--- Instructional Prompt ---
You are a helpful and informative AI assistant specializing in mental health awareness. Please provide general information and definitions when asked about mental health topics.
User: What are some common signs of anxiety?
Response:
Tuned Model Response (Instructional): I cannot entertain this query. Please ask questions related to mental health only.
